In [18]:
# [Cell 1] 모듈 로드
%reload_ext autoreload
%autoreload 2

import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import ipywidgets as widgets
from IPython.display import display
from module.hardware import AGVHardware
from module.driving_logic import LineTrackingBrain
from module.mission_manager import MissionManager

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

# .pth 파일의 경로를 확인
model.load_state_dict(torch.load('./best_steering_model_xy_test.pth'))
device = torch.device('cuda')
model = model.to(device)
model.eval().half()

# 2. 객체 조립
agv = AGVHardware()
brain = LineTrackingBrain(model, device)
manager = MissionManager(agv, brain)

print("시스템 준비 완료!")

⚠️ 카메라가 이미 사용 중이거나 연결되지 않았습니다.
시스템 준비 완료!


In [17]:
agv.stop()

In [7]:
# [Cell 2] UI (슬라이더) 생성 및 Context 연결
# RobotMoving.py에 있던 그 슬라이더들입니다.

speed_slider = widgets.FloatSlider(value=0.25, min=0.0, max=0.5, step=0.01, description='Speed')
steering_gain = widgets.FloatSlider(value=0.1, min=0.0, max=0.2, step=0.001, description='St. Gain')
steering_dgain = widgets.FloatSlider(value=0.25, min=0.0, max=0.5, step=0.001, description='St. DGain')
steering_bias = widgets.FloatSlider(value=0.0, min=-0.3, max=0.3, step=0.01, description='Bias')

image_widget = widgets.Image(format='jpeg', width=224, height=224)
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = widgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')

# 제어 버튼
start_btn = widgets.Button(description='Start Tracking', button_style='success')
stop_btn = widgets.Button(description='Stop', button_style='danger')

def on_start(_): manager.set_state("TRACKING")
def on_stop(_): manager.set_state("IDLE")

start_btn.on_click(on_start)
stop_btn.on_click(on_stop)

display(widgets.HBox([image_widget, y_slider]), x_slider)
display(speed_slider, steering_gain, steering_dgain, steering_bias)
display(widgets.HBox([start_btn, stop_btn]))

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.25, description='Speed', max=0.5, step=0.01)

FloatSlider(value=0.1, description='St. Gain', max=0.2, step=0.001)

FloatSlider(value=0.25, description='St. DGain', max=0.5, step=0.001)

FloatSlider(value=0.0, description='Bias', max=0.3, min=-0.3, step=0.01)

In [5]:
# [Cell 3] 메인 루프를 쓰레드로 분리하여 실행
import threading
import time

# 쓰레드 제어용 플래그
is_running = True

def execution_loop():
    global is_running
    print("🚀 로봇 제어 쓰레드 시작")
    
    while is_running:
        try:
            # 1. 슬라이더 값을 Context에 동기화 (UI -> Logic)
            # 메인 쓰레드가 UI를 담당하므로 여기서 값을 읽어와도 됩니다.
            ctx = manager.context
            ctx.speed_gain = speed_slider.value
            ctx.steering_gain = steering_gain.value
            ctx.steering_dgain = steering_dgain.value
            ctx.steering_bias = steering_bias.value
            
            # 2. FSM 업데이트 (Logic)
            manager.update()
            
            # 3. 화면 업데이트 (Logic -> UI)
            # Context에 처리된 이미지가 있고, 카메라가 켜져있을 때만 업데이트
            if ctx.processed_image:
                image_widget.value = ctx.processed_image
                x_slider.value = float(ctx.current_x) # float 형변환 안전장치
                y_slider.value = float(ctx.current_y)
            elif agv.camera: 
                # IDLE 상태라 처리가 안 되어도 원본 화면은 보여주기 (카메라 미리보기)
                # (주의: bgr8_to_jpeg 함수가 필요합니다. import 확인하세요)
                from jetbot import bgr8_to_jpeg
                image_widget.value = bgr8_to_jpeg(agv.get_frame())

            time.sleep(0.01)
            
        except Exception as e:
            print(f"Error in loop: {e}")
            time.sleep(1) # 에러 나면 1초 쉬고 재시도

# 기존에 돌던 쓰레드가 있다면 정리 (재실행 시 중복 방지)
try:
    if 'thread' in locals() and thread.is_alive():
        is_running = False
        thread.join()
        print("이전 쓰레드 종료됨")
except:
    pass

# 쓰레드 생성 및 시작
is_running = True
thread = threading.Thread(target=execution_loop)
thread.start()

🚀 로봇 제어 쓰레드 시작


In [15]:
# [Cell 4] 강제 종료 (비상 정지)
is_running = False
if 'thread' in locals():
    thread.join()
    
agv.stop()